ETL from australian user items

In [1]:
import pandas as pd
import numpy as np
# from sklearn.linear_model import LinearRegression
# from sklearn.metrics import r2_score
# from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns
import string
import ast

Opening file and store it in DataFrame

In [2]:
# opening file and store it in a dataset

filename = "australian_users_items8200.json"
filepath = "./Datasets/"

# define a temporary list where each line of the file is going to be stored as an element
temp_list = []

with open(filepath + filename, encoding = "utf-8") as aust_user_item_file:
    # iterate over each line inside the file
    for temp_row in aust_user_item_file.readlines():
        # evaluate the file line as a Python sentence and add it as an element to the temporary list
        temp_list.append(ast.literal_eval(temp_row))

# build a dataframe with the temporary list
aust_user_item_df = pd.DataFrame(temp_list)

In [3]:
# no NaNs at sight... yet
aust_user_item_df.isna().sum()

user_id        0
items_count    0
steam_id       0
user_url       0
items          0
dtype: int64

In [4]:
# as 'user_url' doesn't seem to add any disctintive meaningful information, its dropped.
aust_user_item_df.drop(columns="user_url", inplace = True)
# as it is 'steam_id' since the relationship with the others dataframes are on 'user_id'
aust_user_item_df.drop(columns = 'steam_id', inplace = True)

Dropping duplicated rows

In [5]:
# first lets find those rows that have duplicated values in all the columns (except 'item' that gives issues and its not so important right now, since its related to 'items_count' column)
aust_user_item_df[aust_user_item_df.duplicated(list(aust_user_item_df)[:2])]

,user_id,items_count,items
865,bokkkbokkk,0,[]
1732,Nikiad,109,"[{'item_id': '20', 'item_name': 'Team Fortress..."
2343,76561198079743094,48,"[{'item_id': '240', 'item_name': 'Counter-Stri..."
2344,ImSeriouss,50,"[{'item_id': '4000', 'item_name': 'Garry's Mod..."
2394,76561198069124937,0,[]
...,...,...,...
8041,76561198142474106,14,"[{'item_id': '4000', 'item_name': 'Garry's Mod..."
8059,OfficialSenix,45,"[{'item_id': '240', 'item_name': 'Counter-Stri..."
8104,deathjam140,261,"[{'item_id': '4000', 'item_name': 'Garry's Mod..."
8105,cammenball,175,"[{'item_id': '2500', 'item_name': 'Shadowgroun..."


In [6]:
# secondly lets do the same but only looking for duplicated user_id
aust_user_item_df[aust_user_item_df.duplicated(list(aust_user_item_df)[:1])]

,user_id,items_count,items
865,bokkkbokkk,0,[]
1732,Nikiad,109,"[{'item_id': '20', 'item_name': 'Team Fortress..."
2343,76561198079743094,48,"[{'item_id': '240', 'item_name': 'Counter-Stri..."
2344,ImSeriouss,50,"[{'item_id': '4000', 'item_name': 'Garry's Mod..."
2394,76561198069124937,0,[]
...,...,...,...
8041,76561198142474106,14,"[{'item_id': '4000', 'item_name': 'Garry's Mod..."
8059,OfficialSenix,45,"[{'item_id': '240', 'item_name': 'Counter-Stri..."
8104,deathjam140,261,"[{'item_id': '4000', 'item_name': 'Garry's Mod..."
8105,cammenball,175,"[{'item_id': '2500', 'item_name': 'Shadowgroun..."


In [7]:
# by comparing them we can see that all rows with duplicated user_id are duplicated on all the columns, this means that there are no two records with the same user_id but different info
# (e.g. no new entry for an already existing user_id with updated info)
aust_user_item_df[aust_user_item_df.duplicated(list(aust_user_item_df)[:2])].equals(aust_user_item_df[aust_user_item_df.duplicated(list(aust_user_item_df)[:1])])

True

In [8]:
# lets drop duplicates by 'user_id' leaving the newest entry (in case of a future change in database could lead to the above mentioned situation, we keep the updated one)
aust_user_item_df.drop_duplicates(subset = ['user_id'], keep = 'last', inplace = True)

users_number = aust_user_item_df.shape[0]  # number of users

In [9]:
aust_user_item_df

,user_id,items_count,items
0,76561197970982479,277,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,"[{'item_id': '300', 'item_name': 'Day of Defea..."
...,...,...,...
8195,76561198075584298,28,"[{'item_id': '4000', 'item_name': 'Garry's Mod..."
8196,coollord,67,"[{'item_id': '4000', 'item_name': 'Garry's Mod..."
8197,zombiehackerbrah,282,"[{'item_id': '4000', 'item_name': 'Garry's Mod..."
8198,76561198048017700,9,"[{'item_id': '34330', 'item_name': 'Total War:..."


Work on 'items' column. From double nested one to different rows for each game for every user, and expand into columns the info of each game

Dealing with NaNs

In [10]:
# now lets focus on 'item' column which is nested
# expand the nested item column in several rows, one for each game owned for each user
aust_user_item_df = aust_user_item_df.explode('items').reset_index()
aust_user_item_df

,index,user_id,items_count,items
0,0,76561197970982479,277,"{'item_id': '10', 'item_name': 'Counter-Strike..."
1,0,76561197970982479,277,"{'item_id': '20', 'item_name': 'Team Fortress ..."
2,0,76561197970982479,277,"{'item_id': '30', 'item_name': 'Day of Defeat'..."
3,0,76561197970982479,277,"{'item_id': '40', 'item_name': 'Deathmatch Cla..."
4,0,76561197970982479,277,"{'item_id': '50', 'item_name': 'Half-Life: Opp..."
...,...,...,...,...
802338,8199,76561198073904081,160,"{'item_id': '418070', 'item_name': 'Turbo Pug'..."
802339,8199,76561198073904081,160,"{'item_id': '368500', 'item_name': 'Assassin's..."
802340,8199,76561198073904081,160,"{'item_id': '434570', 'item_name': 'Blood and ..."
802341,8199,76561198073904081,160,"{'item_id': '442080', 'item_name': 'Riders of ..."


In [11]:
# now lets focus on NaNs, lets see where they are
aust_user_item_df[aust_user_item_df.isna().any(axis = 1)]

,index,user_id,items_count,items
3733,9,Wackky,0,NaN
3849,11,76561198079601835,0,NaN
6019,31,hellom8o,0,NaN
6523,38,starkillershadow553,0,NaN
7237,54,darkenkane,0,NaN
...,...,...,...,...
796626,8134,76561198064877113,0,NaN
797499,8144,lokery,0,NaN
798798,8162,MT-101,0,NaN
799083,8167,76561198064932358,0,NaN


In [12]:
# in the other hand, lets take a look to those users that don't have any game purchased
aust_user_item_df[aust_user_item_df['items_count'] == 0]

,index,user_id,items_count,items
3733,9,Wackky,0,NaN
3849,11,76561198079601835,0,NaN
6019,31,hellom8o,0,NaN
6523,38,starkillershadow553,0,NaN
7237,54,darkenkane,0,NaN
...,...,...,...,...
796626,8134,76561198064877113,0,NaN
797499,8144,lokery,0,NaN
798798,8162,MT-101,0,NaN
799083,8167,76561198064932358,0,NaN


In [13]:
# so... lets see if there are NaNs that are not coming from users without games purchased
aust_user_item_df[aust_user_item_df.isna().any(axis = 1)].equals(aust_user_item_df[aust_user_item_df['items_count'] == 0])

True

In [14]:
# therefore we're having NaNs in 'items' which relate solely to those users without any game purchased, lets impute those NaNs with a dictionary matching the column structre but with empty values
imputNaN =  {'item_id': '', 'item_name': '', 'playtime_forever': 0, 'playtime_2weeks': 0}
aust_user_item_df['items'] = aust_user_item_df['items'].apply(lambda x: x if x == x else imputNaN)

In [15]:
# and now we can expand the nested 'item' column into several columns
aust_user_item_df = pd.concat([aust_user_item_df.drop(columns='items'), pd.DataFrame(aust_user_item_df['items'].tolist(), index=aust_user_item_df.index).add_prefix('items_')], axis = 1)
aust_user_item_df

,index,user_id,items_count,items_item_id,items_item_name,items_playtime_forever,items_playtime_2weeks
0,0,76561197970982479,277,10,Counter-Strike,6,0
1,0,76561197970982479,277,20,Team Fortress Classic,0,0
2,0,76561197970982479,277,30,Day of Defeat,7,0
3,0,76561197970982479,277,40,Deathmatch Classic,0,0
4,0,76561197970982479,277,50,Half-Life: Opposing Force,0,0
...,...,...,...,...,...,...,...
802338,8199,76561198073904081,160,418070,Turbo Pug,0,0
802339,8199,76561198073904081,160,368500,Assassin's Creed Syndicate,291,0
802340,8199,76561198073904081,160,434570,Blood and Bacon,46,0
802341,8199,76561198073904081,160,442080,Riders of Icarus,118,0


Building user_item dataframe, showing relevant information regarding each user, the games they owns, and how they interact with them (e.g. play hours)

In [16]:
# now, with API's endpoints in mind, lets add release year for each game
game_info = pd.read_csv(filepath + "game_info.csv", sep = "|", encoding = "utf-8")
game_info.rename(columns = {'id': 'items_item_id'}, inplace = True)

aust_user_item_df['items_item_id'] = aust_user_item_df['items_item_id'].apply(lambda x: x if x != '' else '0')
aust_user_item_df['items_item_id'] = aust_user_item_df['items_item_id'].astype(int)
aust_user_item_df = aust_user_item_df.merge(game_info, on = 'items_item_id', how = 'left')

In [17]:
aust_user_item_df

,index,user_id,items_count,items_item_id,items_item_name,items_playtime_forever,items_playtime_2weeks,Unnamed: 0,title,release_year,developer
0,0,76561197970982479,277,10,Counter-Strike,6,0,120416.0,Counter-Strike,2000.0,Valve
1,0,76561197970982479,277,20,Team Fortress Classic,0,0,120413.0,Team Fortress Classic,1999.0,Valve
2,0,76561197970982479,277,30,Day of Defeat,7,0,120424.0,Day of Defeat,2003.0,Valve
3,0,76561197970982479,277,40,Deathmatch Classic,0,0,120418.0,Deathmatch Classic,2001.0,Valve
4,0,76561197970982479,277,50,Half-Life: Opposing Force,0,0,120414.0,Half-Life: Opposing Force,1999.0,Gearbox Software
...,...,...,...,...,...,...,...,...,...,...,...
802338,8199,76561198073904081,160,418070,Turbo Pug,0,0,113322.0,Turbo Pug,2015.0,"Space Cat Studios,SnowFlame,Back To Basics Gaming"
802339,8199,76561198073904081,160,368500,Assassin's Creed Syndicate,291,0,113230.0,Assassin's Creed® Syndicate,2015.0,"Ubisoft Quebec, in collaboration with Ubisoft ..."
802340,8199,76561198073904081,160,434570,Blood and Bacon,46,0,93549.0,Blood and Bacon,2016.0,Big Corporation
802341,8199,76561198073904081,160,442080,Riders of Icarus,118,0,110764.0,Riders of Icarus,2016.0,WeMade


In [18]:
# dropping unnecesary columns
aust_user_item_df.drop(columns = ['Unnamed: 0', 'title'], axis = 1, inplace = True)

In [19]:
# now lets add genres columns, where each game would have an "1" if they're associated with that genre or a "0" if they are not.
# first we need to import the "game - genre matrix"
game_genre_matrix = pd.read_csv(filepath + "game_genres_matrix.csv", delimiter = "|")
game_genre_matrix.rename(columns = {'id': 'items_item_id'}, inplace = True)

# and left join it with our current dataframe on 'items_item_id' column
aust_user_item_df = aust_user_item_df.merge(game_genre_matrix, on = 'items_item_id', how = 'left')
aust_user_item_df.drop(columns = ['Unnamed: 0', 'title'], axis = 1, inplace = True)
aust_user_item_df

,index,user_id,items_count,items_item_id,items_item_name,items_playtime_forever,items_playtime_2weeks,release_year,developer,Action,...,Animation &amp; Modeling,Video Production,Utilities,Web Publishing,Education,Software Training,Design &amp; Illustration,Audio Production,Photo Editing,Accounting
0,0,76561197970982479,277,10,Counter-Strike,6,0,2000.0,Valve,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,76561197970982479,277,20,Team Fortress Classic,0,0,1999.0,Valve,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,76561197970982479,277,30,Day of Defeat,7,0,2003.0,Valve,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,76561197970982479,277,40,Deathmatch Classic,0,0,2001.0,Valve,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,76561197970982479,277,50,Half-Life: Opposing Force,0,0,1999.0,Gearbox Software,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
802338,8199,76561198073904081,160,418070,Turbo Pug,0,0,2015.0,"Space Cat Studios,SnowFlame,Back To Basics Gaming",0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
802339,8199,76561198073904081,160,368500,Assassin's Creed Syndicate,291,0,2015.0,"Ubisoft Quebec, in collaboration with Ubisoft ...",1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
802340,8199,76561198073904081,160,434570,Blood and Bacon,46,0,2016.0,Big Corporation,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
802341,8199,76561198073904081,160,442080,Riders of Icarus,118,0,2016.0,WeMade,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
# changing genre columns to integers in order to reduce memory footprint
list(aust_user_item_df.columns)[9:]
for i in list(aust_user_item_df.columns)[9:]:
    aust_user_item_df[i] = aust_user_item_df[i].fillna(0)
    aust_user_item_df[i] = aust_user_item_df[i].astype(int)

aust_user_item_df

,index,user_id,items_count,items_item_id,items_item_name,items_playtime_forever,items_playtime_2weeks,release_year,developer,Action,...,Animation &amp; Modeling,Video Production,Utilities,Web Publishing,Education,Software Training,Design &amp; Illustration,Audio Production,Photo Editing,Accounting
0,0,76561197970982479,277,10,Counter-Strike,6,0,2000.0,Valve,1,...,0,0,0,0,0,0,0,0,0,0
1,0,76561197970982479,277,20,Team Fortress Classic,0,0,1999.0,Valve,1,...,0,0,0,0,0,0,0,0,0,0
2,0,76561197970982479,277,30,Day of Defeat,7,0,2003.0,Valve,1,...,0,0,0,0,0,0,0,0,0,0
3,0,76561197970982479,277,40,Deathmatch Classic,0,0,2001.0,Valve,1,...,0,0,0,0,0,0,0,0,0,0
4,0,76561197970982479,277,50,Half-Life: Opposing Force,0,0,1999.0,Gearbox Software,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
802338,8199,76561198073904081,160,418070,Turbo Pug,0,0,2015.0,"Space Cat Studios,SnowFlame,Back To Basics Gaming",0,...,0,0,0,0,0,0,0,0,0,0
802339,8199,76561198073904081,160,368500,Assassin's Creed Syndicate,291,0,2015.0,"Ubisoft Quebec, in collaboration with Ubisoft ...",1,...,0,0,0,0,0,0,0,0,0,0
802340,8199,76561198073904081,160,434570,Blood and Bacon,46,0,2016.0,Big Corporation,1,...,0,0,0,0,0,0,0,0,0,0
802341,8199,76561198073904081,160,442080,Riders of Icarus,118,0,2016.0,WeMade,0,...,0,0,0,0,0,0,0,0,0,0


In [50]:
aust_user_item_df.to_csv(filepath + "aust_user_item_df.csv", sep = "|", encoding = 'utf-8')

Building "user - preferred genres" matrix

In [21]:
# first we need to import the "game - genre matrix"
# --------- to improve --> game_genre_matrix is already existing --> check how to reuse it instead importing it again
game_genre_matrix = pd.read_csv(filepath + "game_genres_matrix.csv", delimiter = "|")

# some id cleaning
# game_genre_matrix['id'].replace(34440, int(3900), inplace = True)

# and then build the "user-owned game genres" dataframe
users_genres_df = pd.DataFrame(columns = game_genre_matrix.columns[3:].insert(0, 'user_id'))
users_list = aust_user_item_df['user_id'].unique()          # list of unique users id
users_genres_df['user_id'] = users_list
users_genres_df.fillna(0, inplace = True)
users_genres_df

,user_id,Action,Casual,Indie,Simulation,Strategy,Free to Play,RPG,Sports,Adventure,...,Animation &amp; Modeling,Video Production,Utilities,Web Publishing,Education,Software Training,Design &amp; Illustration,Audio Production,Photo Editing,Accounting
0,76561197970982479,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,js41637,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,evcentric,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Riot-Punch,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,doctr,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8117,76561198075584298,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8118,coollord,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8119,zombiehackerbrah,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8120,76561198048017700,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Expanding aust_user_item dataframe to include the game's genre for each game

In [22]:
user_item_genre = aust_user_item_df
user_item_genre.merge(users_genres_df, on='user_id', how='left')

,index,user_id,items_count,items_item_id,items_item_name,items_playtime_forever,items_playtime_2weeks,release_year,developer,Action,...,Animation &amp; Modeling,Video Production,Utilities,Web Publishing,Education,Software Training,Design &amp; Illustration,Audio Production,Photo Editing,Accounting
0,0,76561197970982479,277,10,Counter-Strike,6,0,2000.0,Valve,1,...,0,0,0,0,0,0,0,0,0,0
1,0,76561197970982479,277,20,Team Fortress Classic,0,0,1999.0,Valve,1,...,0,0,0,0,0,0,0,0,0,0
2,0,76561197970982479,277,30,Day of Defeat,7,0,2003.0,Valve,1,...,0,0,0,0,0,0,0,0,0,0
3,0,76561197970982479,277,40,Deathmatch Classic,0,0,2001.0,Valve,1,...,0,0,0,0,0,0,0,0,0,0
4,0,76561197970982479,277,50,Half-Life: Opposing Force,0,0,1999.0,Gearbox Software,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
802338,8199,76561198073904081,160,418070,Turbo Pug,0,0,2015.0,"Space Cat Studios,SnowFlame,Back To Basics Gaming",0,...,0,0,0,0,0,0,0,0,0,0
802339,8199,76561198073904081,160,368500,Assassin's Creed Syndicate,291,0,2015.0,"Ubisoft Quebec, in collaboration with Ubisoft ...",1,...,0,0,0,0,0,0,0,0,0,0
802340,8199,76561198073904081,160,434570,Blood and Bacon,46,0,2016.0,Big Corporation,1,...,0,0,0,0,0,0,0,0,0,0
802341,8199,76561198073904081,160,442080,Riders of Icarus,118,0,2016.0,WeMade,0,...,0,0,0,0,0,0,0,0,0,0


In [32]:
user_item_genre = aust_user_item_df
user_item_genre.merge(users_genres_df, on='user_id', how='left')
genre_columns_name = game_genre_matrix.columns[3:]      # names of all game's genre available

# some cleaining in 'items_item_id' column
# (e.g. Sid Meier's Civilization IV game have both, 34440 and 3900, ids in user_item_genre but only the later one exist in game_genre DF)
condition = user_item_genre['items_item_id'] == str(34440)
user_item_genre.loc[condition, 'items_item_id'] = '3900'

# "Comparny of Heroes: Opposing Fronts" is not present in game_genre_matrix, an exploration of Comany of Heroes Franchise showed that, lets change its id to the one of "Company of Heroes 2"
game_genre_matrix[game_genre_matrix['title'].str.contains("Company of Heroes", case=True)]
condition = user_item_genre['items_item_id'] == str(9340)
user_item_genre.loc[condition, 'items_item_id'] = 231430
condition = user_item_genre['items_item_id'] == str(23143)
user_item_genre.loc[condition, 'items_item_id'] = 231430
condition = user_item_genre['items_item_id'] == str(228200)
user_item_genre.loc[condition, 'items_item_id'] = 231430

# crysis wars --> crysis warhead
condition = user_item_genre['items_item_id'] == str(17340)
user_item_genre.loc[condition, 'items_item_id'] = 17330

# batman arkham asylum --> batman arkham asylum GOT
condition = user_item_genre['items_item_id'] == str(35010)
user_item_genre.loc[condition, 'items_item_id'] = 35140

# battlefield 2 --> battlefield: bad company 2
condition = user_item_genre['items_item_id'] == str(24860)
user_item_genre.loc[condition, 'items_item_id'] = 24960

# painkiller black edition --> painkiller 2
condition = user_item_genre['items_item_id'] == str(39530)
user_item_genre.loc[condition, 'items_item_id'] = 39560

# and so on other games
condition = user_item_genre['items_item_id'] == str(223530)
user_item_genre.loc[condition, 'items_item_id'] = 550

condition = user_item_genre['items_item_id'] == str(10190)
user_item_genre.loc[condition, 'items_item_id'] = 10180

condition = user_item_genre['items_item_id'] == str(43110)
user_item_genre.loc[condition, 'items_item_id'] = 286690


In [35]:
game_id_list = list(user_item_genre['items_item_id'].unique())
# game_id_list.remove('')
game_id_list = list(map(int, game_id_list))             # convert strings list to integers list


failed_ids = []

counter = 0
# looping on each unique game
for game in game_id_list:

    # creating an empty dict for copying the game's genres being pointed
    temp_dict = dict.fromkeys(genre_columns_name)
    zeros = [0] * len(genre_columns_name)
    for i, col in enumerate(genre_columns_name):
        temp_dict[col] = zeros[i]

    if game in [23120, 10140, 31410, 31220]:
        continue
    # if 
    if len(game_genre_matrix.loc[game_genre_matrix['id'] == game]) == 0:
        failed_ids.append(game)
        continue
    game_index = game_genre_matrix.loc[game_genre_matrix['id'] == game].index[0]
    temp_dict = {key: game_genre_matrix[key].iloc[game_index] for key in game_genre_matrix.columns if key in temp_dict}

    game_repetition = user_item_genre[user_item_genre['items_item_id'] == str(game)].shape[0]

    condition = user_item_genre['items_item_id'] == str(game)

    keys_with_value_1 = [key for key, value in temp_dict.items() if value == 1]

    user_item_genre.loc[condition, keys_with_value_1] = 1
    user_item_genre[user_item_genre['items_item_id'] == str(game)]

    counter += 1
    if counter % 500 == 0:
        print(counter)

500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000


In [36]:
# there're plenty of ids with issue as the ones that were previously corrected
# for the sake of a prompt delivery for this project, we're going to leave them that way
# but it is something that should be addressed for better accuracy of model
len(failed_ids)

1524

In [ ]:
user_item_genre.to_csv(filepath + "user_item_genre.csv", sep = "|", encoding = 'utf-8')

In [ ]:
user_item_genre = pd.read_csv(filepath + "user_item_genre.csv", sep = "|", encoding = 'utf-8')

In [50]:
user_item_genre

,index,user_id,items_count,items_item_id,items_item_name,items_playtime_forever,items_playtime_2weeks,release_year,developer,Action,...,Animation &amp; Modeling,Video Production,Utilities,Web Publishing,Education,Software Training,Design &amp; Illustration,Audio Production,Photo Editing,Accounting
0,0,76561197970982479,277,10,Counter-Strike,6,0,2000.0,Valve,1,...,0,0,0,0,0,0,0,0,0,0
1,0,76561197970982479,277,20,Team Fortress Classic,0,0,1999.0,Valve,1,...,0,0,0,0,0,0,0,0,0,0
2,0,76561197970982479,277,30,Day of Defeat,7,0,2003.0,Valve,1,...,0,0,0,0,0,0,0,0,0,0
3,0,76561197970982479,277,40,Deathmatch Classic,0,0,2001.0,Valve,1,...,0,0,0,0,0,0,0,0,0,0
4,0,76561197970982479,277,50,Half-Life: Opposing Force,0,0,1999.0,Gearbox Software,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
802338,8199,76561198073904081,160,418070,Turbo Pug,0,0,2015.0,"Space Cat Studios,SnowFlame,Back To Basics Gaming",0,...,0,0,0,0,0,0,0,0,0,0
802339,8199,76561198073904081,160,368500,Assassin's Creed Syndicate,291,0,2015.0,"Ubisoft Quebec, in collaboration with Ubisoft ...",1,...,0,0,0,0,0,0,0,0,0,0
802340,8199,76561198073904081,160,434570,Blood and Bacon,46,0,2016.0,Big Corporation,1,...,0,0,0,0,0,0,0,0,0,0
802341,8199,76561198073904081,160,442080,Riders of Icarus,118,0,2016.0,WeMade,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# at this block we're going to multiply each game genre coefficient by a weighting factor to characterize
# particular user interest in that game and later adding them up for each user to summarize each user likeness

# user_genre_temp_df = user_item_genre.drop(user_item_genre.columns[[0,3,4,5]], axis = 1)

user_genre_temp_df = user_item_genre
alpha = 0.5
beta = 0.75

user_genre_matrix = pd.DataFrame(columns = genre_columns_name.insert(0, 'user_id'))

genre_columns_name = game_genre_matrix.columns[3:]      # names of all available game's genres

for i in range(users_number):
    user = users_list[i]
    # filtering dataframe by user
    condition = user_genre_temp_df['user_id'] == user
    user_genre_filtered_df = user_genre_temp_df.copy()      # have to do this in order to prevent some nonsense warnings, idk. Still have some other warnings not clearly related with this code, I gave up trying to understand it, its working
    user_genre_filtered_df = user_genre_filtered_df[condition]

    # weighting purchased games by time played
    for col in genre_columns_name:
        user_genre_filtered_df.loc[:,col] *= (1 + alpha * user_genre_filtered_df.loc[:,'items_playtime_forever'] + beta * user_genre_filtered_df.loc[:,'items_playtime_2weeks'])
    
    # creating an empty dict for copying the game's genres being pointed adding each game genre * weight
    temp_dict = dict.fromkeys(genre_columns_name, 0)
    # filling the dict values wirh the adding up genres values for user
    temp_dict = dict(user_genre_filtered_df.drop(user_genre_filtered_df.columns[[0,1,2,3,]], axis = 1).sum())
    temp_dict['user_id'] = user

    # storing dict in dataframe
    user_genre_matrix.loc[i,:] = temp_dict


In [41]:
user_genre_matrix

,user_id,Action,Casual,Indie,Simulation,Strategy,Free to Play,RPG,Sports,Adventure,...,Animation &amp; Modeling,Video Production,Utilities,Web Publishing,Education,Software Training,Design &amp; Illustration,Audio Production,Photo Editing,Accounting
0,76561197970982479,62810.5,1382.0,9698.5,10844.0,21481.5,1539.5,19895.5,484.0,9574.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,js41637,40338.75,11646.25,32405.75,9489.5,13827.0,725.0,12398.5,1406.5,31872.75,...,0.0,0.0,122.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,evcentric,34620.75,19322.5,37729.5,21363.75,24338.75,5061.25,25647.5,707.0,7566.75,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Riot-Punch,32454.75,765.0,3120.5,1359.5,1419.0,27.5,4192.0,595.0,6895.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,doctr,103389.0,2132.0,13027.5,9580.0,20963.5,5833.0,33744.75,3546.5,37611.25,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8117,76561198075584298,16298.5,222.5,3079.5,2819.5,42.0,343.5,72.0,25.5,6856.75,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8118,coollord,27743.0,1940.0,13064.5,11016.5,8158.0,7112.5,11711.5,22.5,13752.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8119,zombiehackerbrah,55813.25,2265.5,19117.0,14827.5,13786.0,4868.0,12923.0,3262.0,28266.75,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8120,76561198048017700,2073.5,1712.5,1223.5,0.0,837.5,1714.5,2533.0,0.0,820.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [42]:
# now lets normalize features row by row (i.e. normalize the distribution of hours played on each genre by an user)
# for doing that lets define a function that normalize a row's features from the user_genre_matrix

def normalize_row(vector):      # come back later to here, maybe redoing it for making it more general
    vector_norm = np.sqrt(np.sum(genre_vector ** 2))
    if vector_norm != 0:
        genre_vector_norm = genre_vector / vector_norm
    else:
        genre_vector_norm = genre_vector
    
    dict_normalized_row = dict(genre_vector_norm)
    dict_normalized_row['user_id'] = user
    return dict_normalized_row

# building the normalized version of 'user_genre_matrix'
user_genre_mat_norm = pd.DataFrame(columns = genre_columns_name.insert(0, 'user_id'))

for i in range(users_number):
    # temp_dict = temp_dict.fromkeys(temp_dict, 0)
    user = user_genre_matrix['user_id'][i]
    genre_vector = user_genre_matrix.loc[i,:][1:]

    user_genre_mat_norm.loc[i,:] = normalize_row(genre_vector)

user_genre_mat_norm




,user_id,Action,Casual,Indie,Simulation,Strategy,Free to Play,RPG,Sports,Adventure,...,Animation &amp; Modeling,Video Production,Utilities,Web Publishing,Education,Software Training,Design &amp; Illustration,Audio Production,Photo Editing,Accounting
0,76561197970982479,0.87427,0.019236,0.134995,0.150939,0.299005,0.021429,0.276929,0.006737,0.133262,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,js41637,0.617189,0.178189,0.495813,0.145191,0.211555,0.011093,0.189699,0.02152,0.487658,...,0.0,0.0,0.001867,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,evcentric,0.499508,0.278785,0.544361,0.308236,0.351159,0.073024,0.370042,0.010201,0.109173,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Riot-Punch,0.963891,0.02272,0.092677,0.040377,0.042144,0.000817,0.124501,0.017671,0.204793,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,doctr,0.873795,0.018019,0.110102,0.080966,0.177174,0.049298,0.285195,0.029973,0.317873,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8117,76561198075584298,0.887761,0.012119,0.167737,0.153575,0.002288,0.01871,0.003922,0.001389,0.37348,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8118,coollord,0.693293,0.04848,0.32648,0.2753,0.203867,0.17774,0.292668,0.000562,0.34366,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8119,zombiehackerbrah,0.793943,0.032227,0.271939,0.210921,0.196106,0.069247,0.183829,0.046402,0.402094,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8120,76561198048017700,0.438186,0.361897,0.258558,0.0,0.176986,0.36232,0.535291,0.0,0.173394,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
user_genre_mat_norm.to_csv(filepath + "user_genre_mat_norm.csv", sep = "|", encoding = "utf-8")

Expanding user-items dataframe to include each game genre

In [ ]:
aust_user_item_df.columns.insert()

In [ ]:
# -------------------------------------------------------------- ALTERNATE ------------------------------------------------------
# now lets fill the dataframe in the following way:
# for each user:
#      - search for every game owned and look for each of them its genre vector and multiply it by (1 + alpha * 'items_playtime_forever' + beta * 'items_playtime_2week')
#      - looping over all games owned, adding vector element-wise
#      - resulting vector is added to users_genres_df for that particular user
# temp_dict = dict.fromkeys(game_genre_matrix.columns[3:].insert(0, 'user_id'))
alpha = 0.5
beta = 0.75
user_id_index_game_genres = 0           # counter for following the index in users_genres_df

for user in users_list:
    # extracting info from games owned by user
    game_list = list(aust_user_item_df[aust_user_item_df['user_id'] == user]['items_item_name'])

    if game_list[0] == '':
        continue

    game_id_list = list(aust_user_item_df[aust_user_item_df['user_id'] == user]['items_item_id'])
    game_id_list = list(map(int, game_id_list))   # convert it to integer, since it was a list of strings
    hours_played_list = list(aust_user_item_df[aust_user_item_df['user_id'] == user]['items_playtime_forever'])
    hours_played2w_list = list(aust_user_item_df[aust_user_item_df['user_id'] == user]['items_playtime_2weeks'])  

    # creating an empty dict for counting the game's genres owned by user
    counter_dict = dict.fromkeys(game_genre_matrix.columns[3:])
    zeros = [0] * len(game_genre_matrix.columns[3:])
    for i, col in enumerate(game_genre_matrix.columns[3:]):
        counter_dict[col] = zeros[i]

    game_for_user_counter = 0

    # looping inside user's game list to extract its vector of game genre "likeness"
    for game in game_id_list:
        game_for_user_counter += 1
        # extracting genres corresponding to game and store it in a dictionary
        if len(game_genre_matrix.loc[game_genre_matrix['id'] == game]) == 0:
            continue
        
        game_index = game_genre_matrix.loc[game_genre_matrix['id'] == game].index[0]
        game_id = game_genre_matrix.loc[game_index, 'id']
        game_hs_played = hours_played_list[game_for_user_counter - 1]
        game_hs_2w_played = hours_played2w_list[game_for_user_counter - 1]

        previous_dict = counter_dict
        pickedup_dict = {key: game_genre_matrix[key].iloc[game_index] for key in game_genre_matrix.columns if key in counter_dict}

        # multiply the values of game genres by a factor that enfatize the hours played
        pickedup_dict.update((key, value * (1 + alpha * game_hs_played + beta * game_hs_2w_played)) for key, value in pickedup_dict.items())
        # add the updated genres vector of this particular game to the accumulated of every other game already accounted for this user
        counter_dict = {k: previous_dict[k] + pickedup_dict[k] for k in previous_dict}
    
    # updating the matrix
    counter_dict['user_id'] = user
    users_genres_df.loc[user_id_index_game_genres, :] = counter_dict
    user_id_index_game_genres += 1



In [ ]:
users_genres_df.to_csv(filepath + "users_genres_df.csv", sep = "|", encoding = 'utf-8')

Building "user - owned game" matrix

In [46]:
temp_matrix_col = list(aust_user_item_df['items_item_name'].unique())
temp_matrix_col.insert(0, 'user_id')

In [48]:
users_games_matrix = pd.DataFrame(columns = temp_matrix_col)
users_games_matrix['user_id'] = aust_user_item_df['user_id'].unique()
users_games_matrix.fillna(0, inplace = True)

In [ ]:
aust_user_item_df[aust_user_item_df['index'] == 8].items_count.iloc[0]

In [ ]:
max_index = aust_user_item_df['index'].max()
for i in range(0, max_index):
    if aust_user_item_df[aust_user_item_df['index'] == i].items_count.any() == 0:
        continue

    game_each_user = list(aust_user_item_df[aust_user_item_df['index'] == i].iloc[:,4])
    id_each_user = aust_user_item_df[aust_user_item_df['index'] == i].iloc[:,1].iloc[0]
    users_games_matrix.iloc[i, 0] = id_each_user
    for j in game_each_user:
        temp_game_col = users_games_matrix.columns.get_loc(j)
        users_games_matrix.iloc[i, temp_game_col] = 1


In [ ]:
max_index

In [ ]:
users_games_matrix

In [ ]:
aust_user_item_df[aust_user_item_df['index'] == 1]

ETL process finished

Starting EDA

In [ ]:
# lets start by seeing how many users doesn't own any game
print("proportion of users with 0 games: ", aust_user_item_df[aust_user_item_df['items_count'] == 0].shape[0] / users_number)

# and which is the maximum amount of games owned by a single user
print("max games owned by a single user: ", aust_user_item_df.items_count.max())

In [ ]:
# now lets see the distribution of games owned by users, using a log scale in games owned due to its large spread and dropping those users that have 0 games (to deal with the log transformation)
prueba = aust_user_item_df[['user_id', 'items_count']].drop_duplicates()
prueba = prueba[prueba['items_count'] > 0]
prueba['items_count'] = np.log10(prueba['items_count'])

ax = sns.histplot(data = prueba, x = 'items_count', kde = True)
ax.lines[0].set_color('crimson')
ax.set(xlabel='Log10(games bought)', ylabel='Users count')

# which looks surprisingly normally distributed (in the log scale)

In [ ]:
# how many different games are in the data frame
print("amount of different games: ", len(aust_user_item_df['items_item_name'].unique()))

# lets see how many times were bought each game
game_occurence = aust_user_item_df.groupby(['items_item_name']).size().sort_values(ascending = False)
game_occurence

In [ ]:
game_occurence[1]

In [ ]:
# lets see which proportion of purchased games were never played by its owner
aust_user_item_df[aust_user_item_df['items_playtime_forever'] == 0].shape[0] / aust_user_item_df.shape[0]

# seems a pretty considerable number so it should be taken into account at the moment of building the recommendation algorithm (e.g. owner liked it enough to bought it but found not enough attractive to play with it yet)

In [ ]:
# redoing the analysis but now looking for the fortnight before window the proportion of unplayed games by its owner skyrocketed
aust_user_item_df[aust_user_item_df['items_playtime_2weeks'] == 0].shape[0] / aust_user_item_df.shape[0]

# therefore we can do a weighted recommendation based on, for example, 'game owned and never played' weight 1, 'game owned and played' weight 2, 'game owned and played recently' weight 4
# something like weight = 1 + 2 * (if played forever != 0) + 4 * (if played 2 weeks != 0)
# ------------> retake this idea, maybe these columns should be standarized <------------------